In [7]:
import sys, warnings, io
import pandas as pd

from PyQt5.QtGui import QFont
from PyQt5.QtCore import (Qt, QAbstractTableModel, QVariant, QModelIndex)
from PyQt5.QtWidgets import (QDialog, QApplication, QPushButton, QGroupBox, QVBoxLayout,
                             QLabel, QFileDialog, QComboBox, QGridLayout, QTextEdit,
                             QStyleFactory, QCheckBox, QHBoxLayout, QMessageBox, QLineEdit,
                             QTableView, QMessageBox)

warnings.filterwarnings('ignore')

### Table Model

In [8]:
class PandasModel(QAbstractTableModel):
    def __init__(self, df = pd.DataFrame(), parent = None):
        QAbstractTableModel.__init__(self, parent = parent)
        self._df = df
        
    def headerData(self, section, orientation, role = Qt.DisplayRole):
        if role != Qt.DisplayRole:
            return QVariant()
        
        if orientation == Qt.Horizontal:
            try:
                return self._df.columns.tolist()[section]
            except (IndexError, ):
                return QVariant()
            
    def data(self, index, role = Qt.DisplayRole):
        if role != Qt.DisplayRole:
            return QVariant()
        
        if not index.isValid():
            return QVariant()
        
        return QVariant(str(self._df.iloc[index.row(), index.column()]))
    
    def setData(self, index, value, role):
        row = self._df.index[index.row()]
        col = self._df.index[index.column()]
        
        if hasattr(value, 'toPyObject'):
            value = value.toPyObject()
            
        else:
            dtype = self._df[col].dtype
            if dtype != object:
                value = None if value == '' else dtype.type(value)
        
        self._df.set_value(row, col, value)
        return True
    
    def rowCount(self, parent = QModelIndex()):
        return len(self._df.index)
    
    def columnCount(self, parent = QModelIndex()):
        return len(self._df.columns)
    
    def sort(self, column, order):
        colname = self._df.columns.tolist()[column]
        self.layoutAboutToBeChanged.emit()
        self._df.sort_values(colname, ascending = order == Qt.AscendingOrder, inplace = True)
        self._df.reset_index(inplace = True, drop = True)
        self.layoutChanged.emit()

### App Widgets

In [9]:
class TableConverter(QDialog):
    def __init__(self, parent = None):
        super(TableConverter, self).__init__(parent)
        
        # set window title
        self.setWindowTitle('Table Converter')
        
        # window palette
        self.originalPalette = QApplication.palette()
        QApplication.setStyle(QStyleFactory.create('Fusion'))
        
        # window size and minimize
        self.setFixedSize(800, 600)
        self.setWindowFlag(Qt.WindowType.WindowMinimizeButtonHint)
        self.setWindowFlag(Qt.WindowType.WindowMaximizeButtonHint)
        
        # grouping widget
        self.createTopGroupBox()
        self.createBottomLeftGroupBox()
        self.createBottomRightGroupWidget()
        
        # set font size for all widgets
        font = QFont()
        font.setPointSize(9)
        font.setFamily('Segoe UI')
        
        self.setFont(font)
        self.topGroupBox.setFont(font)
        self.bottomLeftGroupBox.setFont(font)
        self.bottomRightGroupWidget.setFont(font)
        
        # main layout
        mainLayout = QGridLayout()
        mainLayout.addWidget(self.topGroupBox, 1, 0, 1, 2)
        mainLayout.addWidget(self.bottomLeftGroupBox, 2, 0, 1, 1)
        mainLayout.addWidget(self.bottomRightGroupWidget, 2, 1, 1, 1)
        mainLayout.setRowStretch(1, 1)
        mainLayout.setRowStretch(2, 1)
        mainLayout.setColumnStretch(0, 1)
        mainLayout.setColumnStretch(1, 1)
        self.setLayout(mainLayout)
        
        # dataframe variable
        self.df = None
    
    # ==== Top Group Box =====
    def createTopGroupBox(self):
        self.topGroupBox = QGroupBox()
        self.pathLine = QLineEdit()
        self.importComboBox = QComboBox()
        self.loadBtn = QPushButton('Select File')
        self.pandasTv = QTableView(self)
        
        # load combo box item
        import_formats = [
            'Excel File (*.xlsx)',
            'CSV File (*.csv)',
            'SAS File (*.sas7bdat)',
            'Pickle File (*.pkl)'
        ]
        self.importComboBox.addItems(import_formats)
        
        # actiate non-editable line
        self.pathLine.setReadOnly(True)
        
        # connect the button
        self.loadBtn.clicked.connect(self.loadFile)
        
        # sort the table
        self.pandasTv.setSortingEnabled(True)
        
        # layout
        VLayout = QVBoxLayout()
        HLayout = QHBoxLayout()
        HLayout.addWidget(self.pathLine)
        HLayout.addWidget(self.importComboBox)
        HLayout.addWidget(self.loadBtn)
        VLayout.addLayout(HLayout)
        VLayout.addWidget(self.pandasTv)
        self.topGroupBox.setLayout(VLayout)
        
    # ===== Bottom Left Group Box ======
    def createBottomLeftGroupBox(self):
        self.bottomLeftGroupBox = QGroupBox('Table Info')
        self.infoLine = QTextEdit()
        
        # activate non-editable line
        self.infoLine.setReadOnly(True)
        
        # layout
        VLayout = QVBoxLayout()
        VLayout.addWidget(self.infoLine)
        self.bottomLeftGroupBox.setLayout(VLayout)
        
    # ===== Bottom Right Group Widget ======
    def createBottomRightGroupWidget(self):
        self.bottomRightGroupWidget = QGroupBox('Table Export')
        self.bottomRightGroupWidget.setCheckable(True)
        self.bottomRightGroupWidget.setChecked(False)
        self.pathLine1 = QLineEdit()
        self.pathLabel = QLabel('&Output Path:')
        self.pathLabel.setBuddy(self.pathLine1)
        self.exportComboBox = QComboBox()
        self.exportBtn = QPushButton('Export')
        
        # export combo box
        export_formats = [
            'Excel File (*.xlsx)',
            'CSV File (*.csv)',
            'Text File (*.txt)',
            'Pickle File (*.pkl)'
        ]
        self.exportComboBox.addItems(export_formats)
        
        # activate non-editable line
        self.pathLine1.setReadOnly(True)
        
        # connect the button
        self.exportBtn.clicked.connect(self.saveFile)
        
        # layout
        layout = QGridLayout()
        layout.addWidget(self.exportComboBox, 0, 1, 1, 2)
        layout.addWidget(self.exportBtn, 0, 0, 1, 1)
        layout.addWidget(self.pathLabel, 1, 0, 1, 1)
        layout.addWidget(self.pathLine1, 1, 1, 1, 2)
        layout.setRowStretch(2, 1)
        layout.setColumnStretch(2, 1)
        self.bottomRightGroupWidget.setLayout(layout)
    
    # ==== Load File ======
    def loadFile(self):
        selected_format = self.importComboBox.currentText()
        fileName, _ = QFileDialog.getOpenFileName(self, 'Open File', '', selected_format)
        
        # SAS file
        if selected_format == 'SAS File (*.sas7bdat)':
            if fileName:
                self.df = pd.read_sas(fileName, encoding = 'utf-8')
                model = PandasModel(self.df)
                
                # table info
                buf = io.StringIO()
                self.df.info(buf = buf)
                lines = buf.getvalue().splitlines()
                df_info = pd.DataFrame([x.split() for x in lines[5:-2]], columns=lines[3].split())
                df_info = df_info.drop('Count',axis=1)
                df_info = df_info.rename(columns={'Non-Null':'Non-Null Count'})
                
                # save to layout
                self.pandasTv.setModel(model)
                self.pathLine.setText(fileName)
                self.infoLine.setText(df_info)
                
                # activate non-editable line
                self.pathLine.setReadOnly(True)
                self.infoLine.setReadOnly(True)
        
        # Excel file
        elif selected_format == 'Excel File (*.xlsx)':
            if fileName:
                self.df = pd.read_excel(fileName)
                model = PandasModel(self.df)
                
                # table info
                buf = io.StringIO()
                self.df.info(buf = buf)
                lines = buf.getvalue().splitlines()
                df_info = pd.DataFrame([x.split() for x in lines[5:-2]], columns=lines[3].split())
                df_info = df_info.drop('Count',axis=1)
                df_info = df_info.rename(columns={'Non-Null':'Non-Null Count'})
                
                # save to layout
                self.pandasTv.setModel(model)
                self.pathLine.setText(fileName)
                self.infoLine.setText(df_info)
                
                # activate non-editable line
                self.pathLine.setReadOnly(True)
                self.infoLine.setReadOnly(True)
                
        # CSV file
        elif selected_format == 'CSV File (*.csv)':
            if fileName:
                self.df = pd.read_csv(fileName)
                model = PandasModel(self.df)
                
                # table info
                buf = io.StringIO()
                self.df.info(buf = buf)
                df_info = buf.getvalue()
                
                # save to layout
                self.pandasTv.setModel(model)
                self.pathLine.setText(fileName)
                self.infoLine.setText(df_info)
                
                # activate non-editable line
                self.pathLine.setReadOnly(True)
                self.infoLine.setReadOnly(True)
                
        # Pickle file
        elif selected_format == 'Pickle File (*.pkl)':
            if fileName:
                self.df = pd.read_pickle(fileName)
                model = PandasModel(self.df)
                
                # table info
                buf = io.StringIO()
                self.df.info(buf = buf)
                df_info = buf.getvalue()
                
                # save to layout
                self.pandasTv.setModel(model)
                self.pathLine.setText(fileName)
                self.infoLine.setText(df_info)
                
                # activate non-editable line
                self.pathLine.setReadOnly(True)
                self.infoLine.setReadOnly(True)
    
    # ===== Save File =====
    def saveFile(self):
        selected_format = self.exportComboBox.currentText()
        
        # Excel file
        if selected_format == 'Excel File (*.xlsx)':
            fileName, _ = QFileDialog.getSaveFileName(self, 'Save As', '', 'Excel File (*.xlsx)')
            if fileName:
                self.df.to_excel(fileName, index = False)
                self.pathLine1.setText(fileName)
                self.pathLine1.setReadOnly(True)
                QMessageBox.about(self, 'Information', 'File has been saved successfully.')
        
        # CSV file
        elif selected_format == 'CSV File (*.csv)':
            fileName, _ = QFileDialog.getSaveFileName(self, 'Save As', '', 'CSV File (*.csv)')
            if fileName:
                self.df.to_csv(fileName, index = False)
                self.pathLine1.setText(fileName)
                self.pathLine1.setReadOnly(True)
                QMessageBox.about(self, 'Information', 'File has been saved successfully.')
        
        # Text file
        elif selected_format == 'Text File (*.txt)':
            fileName, _ = QFileDialog.getSaveFileName(self, 'Save As', '', 'Text File (*.txt)')
            if fileName:
                with open(fileName, 'a') as f:
                    df_string = self.df.to_string(header = True, index = False)
                    f.write(df_string)
                self.pathLine1.setText(fileName)
                self.pathLine1.setReadOnly(True)
                QMessageBox.about(self, 'Information', 'File has been saved successfully.')
        
        # Pickle file
        elif selected_format == 'Pickle File (*.pkl)':
            fileName, _ = QFileDialog.getSaveFileName(self, 'Save As', '', 'Pickle File (*.pkl)')
            if fileName:
                self.df.to_pkl(fileName, index = False)
                self.pathLine1.setText(fileName)
                self.pathLine1.setReadOnly(True)
                QMessageBox.about(self, 'Information', 'File has been saved successfully.')

                
def main():
    app = QApplication(sys.argv)
    window = TableConverter()
    window.setGeometry(100, 100, 1000, 800)
    window.show()
    sys.exit(app.exec_())
    
if __name__ == '__main__':
    main()

SystemExit: 0